## Environment Setup

In [3]:
import pandas as pd
pd.options.mode.chained_assignment = None

import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
#import plotly.express as px

import warnings
warnings.filterwarnings('ignore')


import sys
sys.path.insert (0, './_functions_/')
from functions_electric_cars import *

ModuleNotFoundError: No module named 'plotly'

## Datasets

In [2]:
df = merge_excel_sheets("./raw_data/02__autos_nach_bundesland_kraftstoff.xlsx")
df.head()

NameError: name 'merge_excel_sheets' is not defined

## Handling missing values

Check for NaNs

In [ ]:
df.isnull().sum()

The lines that contain no Kraftstoffart are the ones that include totals, therefore we will replace with the value 'Total'

In [ ]:
# the lines that contain no Kraftstoffart are the ones that include totals, therefore we will replace with the value 'Total'
df['Kraftstoffart'] = df['Kraftstoffart'].fillna('Total') 
df

In [ ]:
#remove the text "zusammen" or "insgesamt" from the Land
df['Land'] = df['Land'].str.replace(r'\s+zusammen$', '', regex=True)
df['Land'] = df['Land'].str.replace(r'\s+insgesamt$', '', regex=True)


In [ ]:
# Remove "Sonstige" from the dataset,since we are interested in the german states
df = df[~(df["Land"] =="Sonstige")]
df

In [ ]:
df ['Kraftstoffart'].value_counts()

In [ ]:
df = df.rename(columns ={'Land':'land', 'Kraftstoffart':'fuel', 'Kraftfahrzeuge \ninsgesamt': 'total_cars', 'Year':'year'})
df

In [ ]:
# harmonize the types of "fuel"

fuel_type_mapping = {
 'Benzin': 'Benzin', 
 'Diesel': 'Diesel', 
 'Gas insgesamt': 'Gas', 
 'Hybrid insgesamt': 'Hybrid total', 
 'Sonstige': 'Sonstige',
 'Elektro (BEV)': 'Electric', 
 'Elektro': 'Electric', 
 'darunter Plug-in': 'Plug-in Hybrid',
 '  darunter Plug-in': 'Plug-in Hybrid'  
 }

df['fuel'] = df['fuel'].replace(fuel_type_mapping)
df["fuel"].value_counts()


In [ ]:
df.head()

In [ ]:
# one df for deutschland
df_germany = df [df ["land"] =="Deutschland"].reset_index()
df_germany.head(8)

In [ ]:
df_states = df [~(df ["land"] =="Deutschland")].reset_index()
df_states

In [ ]:
# we consider electric and hybrid as electrical, since they all use the charging stations
electric_types = ["Electric", "Plug-in Hybrid"]

# filter the electrical vehicles
eautos = df_germany [df_germany["fuel"].isin(electric_types)]
# summs the electrical vehicles
e_autos_year = eautos.groupby("year").agg(
    number_of_electric_cars=("total_cars", "sum")     
).reset_index()

e_autos_year

In [ ]:
charging_stations_germany = pd.read_csv('./clean_data/stations_germany.csv', usecols= ['number_of_charging_points', 'year'])

charging_stations_germany = charging_stations_germany [charging_stations_germany["year"] > 2018]

charging_stations_germany = charging_stations_germany.merge(e_autos_year, on = "year", how = "left")

charging_stations_germany

In [ ]:
plt.figure(figsize=(8, 6))
sns.lineplot(data = charging_stations_germany, x ='year', y ='number_of_charging_points', label='Charging Stations', color = "blue")
sns.lineplot(data = charging_stations_germany, x ='year', y = 'number_of_electric_cars', label='Electric Cars', color = "limegreen")
sns.despine()

plt.xlabel('Year')
plt.ylabel('Count')


plt.title('Growth of Charging Stations vs Electric Cars')

# Move legend outside
plt.legend(title="Charging points x Electric cars", bbox_to_anchor=(1.05, 1), loc="upper left")
plt.show()


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Assuming your DataFrame is called df
# and has columns: 'year', 'number of electric cars', 'total number of cars'

# Sort by year in case it's not sorted
df = df.sort_values('year')

# Compute the number of new cars and new electric cars per year
df['new_total_cars'] = df['total_number_of_cars'].diff()
df['new_electric_cars'] = df['number_of_electric_cars'].diff()

# Compute percentage of new electric cars
df['percent_new_electric'] = (df['new_electric_cars'] / df['new_total_cars']) * 100

# Drop the first row with NaN from diff
df = df.dropna()

# Plotting
plt.figure(figsize=(10, 6))
plt.plot(df['year'], df['percent_new_electric'], marker='o')
plt.title('Percentage of New Electric Cars per Year')
plt.xlabel('Year')
plt.ylabel('Percentage (%)')
plt.grid(True)
plt.show()


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
import numpy as np

df = charging_stations_germany

# Train linear regression on year vs number_of_electric_cars
X = df[['year']]
y = df['number_of_electric_cars']

model = LinearRegression()
model.fit(X, y)

# Predict up to 2030
future_years = np.arange(2019, 2031).reshape(-1, 1)
future_predictions = model.predict(future_years)

# Plotting
plt.figure(figsize=(10, 6))
plt.plot(df['year'], df['number_of_electric_cars'], 'o', label='Actual')
plt.plot(future_years.flatten(), future_predictions, '-', label='Predicted')

plt.title("Linear Regression: Number of Electric Cars until 2030")
plt.xlabel("Year")
plt.ylabel("Number of Electric Cars")
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()


In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from matplotlib.ticker import ScalarFormatter

from matplotlib.ticker import FuncFormatter

# Function to convert large numbers to 'M' format
def millions_formatter(x, pos):
    return f'{x*1e-6:.1f}M' if x >= 1e6 else f'{int(x)}'
    

df = charging_stations_germany

# Set up polynomial regression (degree 2)
poly = PolynomialFeatures(degree=2)
X_poly = poly.fit_transform(df[['year']])
y = df['number_of_electric_cars']

model = LinearRegression()
model.fit(X_poly, y)

# Predict up to 2030
future_years = np.arange(2019, 2031).reshape(-1, 1)
future_X_poly = poly.transform(future_years)
future_predictions = model.predict(future_X_poly)

# Plotting
plt.figure(figsize=(10, 6))
plt.plot(df['year'], df['number_of_electric_cars'], 'o', label='Actual')
plt.plot(future_years.flatten(), future_predictions, '-', label='Polynomial Fit (degree=2)')

plt.gca().yaxis.set_major_formatter(FuncFormatter(millions_formatter))

plt.title("Polynomial Regression: Number of Electric Cars until 2030")
plt.xlabel("Year")
plt.ylabel("Number of Electric Cars")
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
# Create the scatter plot
plt.figure(figsize=(8, 6))
plt.scatter(charging_stations_germany['number_of_electric_cars'], charging_stations_germany['number_of_charging_points'], color='green')
plt.title('Correlation between Electric Cars and Charging Points')
plt.xlabel('Number of Electric Cars')
plt.ylabel('Number of Charging Points')
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
correlation = charging_stations_germany['number_of_electric_cars'].corr(charging_stations_germany['number_of_charging_points'])
print(f'Correlation: {correlation:.2f}')

In [ ]:
# Set the plot style
sns.set_style("whitegrid")

# excludes total, as it includes all the fuel types
df_germany_filtered = df_germany[~(df_germany["fuel"].isin(["Total","Sonstige"]))]

# Pivot the data to get years on x-axis and fuel types as separate lines
plt.figure(figsize=(12, 6))
sns.lineplot(data=df_germany_filtered, x="year", y="total_cars", palette = colors_fuel, hue="fuel", marker="o")

# Access the axes
ax = plt.gca()

# Turn off scientific notation
ax.ticklabel_format(style='plain', axis='y')

# Format ticks with commas
ax.yaxis.set_major_formatter(ticker.FuncFormatter(lambda x, _: f'{int(x):,}'.replace(',', '.')))

# Labels and title
plt.xlabel("Year")
plt.ylabel("Number of Cars")
plt.title("Cars Over Years (2019-2025) in Germany")

# Move legend outside
plt.legend(title="Fuel Type", bbox_to_anchor=(1.05, 1), loc="upper left")

# Adjust layout to fit legend
plt.tight_layout()
plt.show()

In [ ]:
import matplotlib.ticker as ticker

# here we consider pure electrical and plug-in hybrids as electrical cars
df_electric = df_germany[df_germany["fuel"].isin(["Electric", "Plug-in Hybrid"])]

# Set the plot style
sns.set_style("whitegrid")


# Pivot the data to get years on x-axis and fuel types as separate lines
plt.figure(figsize=(12, 6))

# Access the axes
ax = plt.gca()

# Turn off scientific notation
ax.ticklabel_format(style='plain', axis='y')

# Format ticks with commas
ax.yaxis.set_major_formatter(ticker.FuncFormatter(lambda x, _: f'{int(x):,}'.replace(',', '.')))


sns.lineplot(data=df_electric, x="year", y="total_cars", palette = colors_fuel, hue="fuel", marker="o", ax = ax)

# Labels and title
plt.xlabel("Year")
plt.ylabel("Number of Cars")
plt.title("Electrict Vehicles in Germany over the Years (2019-2025)")

# Move legend outside
plt.legend(title="Fuel Type", bbox_to_anchor=(1.05, 1), loc="upper left")

# Adjust layout to fit legend
plt.tight_layout()
plt.show()

In [ ]:
# Remove "Deutschland" from the dataset, as it contains aggregations
df_states = df[~(df["land"] =="Deutschland")]
df_states

In [ ]:
df_states.to_csv('./clean_data/cars_based_on_fuel_per_year.csv',  index=False)

Since we are only interested in E-Autos and the rest, we will convert the data again

In [ ]:
fuel_type_mapping2 = {
 'Benzin': 'Others', 
 'Diesel': 'Others', 
 'Gas': 'Others',  
 'Sonstige': 'Others',
 'Hybrid': 'Others',  
 'Electric': 'Electric',
 'Hybrid E': 'Plug-in Hybrid'  
 }

df_states['fuel'] = df_states['fuel'].replace(fuel_type_mapping2)
df_states["fuel"].value_counts()

In [ ]:
df_states.to_csv('./clean_data/cars_based_on_fuel_per_year_filtered.csv',  index=False)

In [ ]:
# Filter data for 2025
df_2025 = df_states[df_states["year"] == 2025]

# Sort by total number of cars for better visualization
df_2025 = df_2025.sort_values(by="total_cars", ascending=False)

# Create the bar plot
plt.figure(figsize=(12, 6))
sns.barplot(data=df_2025, x="total_cars", y="land", palette=german_states_colors)

# Labels and title
plt.xlabel("Total Cars")
plt.ylabel("Land")
plt.title("Total Number of Cars per State (2025)")

# Show the plot
plt.show()


In [ ]:
df_states['fuel'].value_counts()


In [ ]:
# Filter data for 2025
df_2025 = df_states[df_states["year"] == 2025]

# Pivot table to get total cars by fuel type for each Land
df_pivot = df_2025.pivot_table(index="land", columns="fuel", values="total_cars", aggfunc="sum")

# Sort by total number of cars for better visualization
df_pivot = df_pivot.sort_values(by="Total", ascending=True)

# Handle missing values (if some states don't have electric cars)
df_pivot = df_pivot.fillna(0)

# Compute non-electric cars
df_pivot["Non_Electric"] = df_pivot["Total"] - df_pivot["Electric"]

# Plot stacked bar chart
plt.figure(figsize=(12, 6))

plt.barh(df_pivot.index, df_pivot["Non_Electric"], color="lightgray", label="Other Fuels")
plt.barh(df_pivot.index, df_pivot["Electric"], left=df_pivot["Non_Electric"], color="#81E552", label="Electric")

# Labels and title
plt.xlabel("Total Cars (in 10 milions)")
plt.ylabel("State")
plt.title("Total Number of Cars per State (2025)")
plt.legend()

# Show the plot
plt.show()



### Creates a new dataframe

Creates a Dataframe with the information about the year 2025

In [ ]:
autos_2025 = df_states [df_states["year"] == 2025]
autos_2025.rename(columns = {"land":"state"}, inplace=True)

# we consider electric and hybrid as electrical, since they all use the charging stations
electric_types = ["Electric", "Plug-in Hybrid"]

# filter the electrical vehicles
eautos_2025 = autos_2025[autos_2025["fuel"].isin(electric_types)]
# summs the electrical vehicles
e_autos_per_state_2025 = eautos_2025.groupby("state").agg(
    number_of_electric_cars=("total_cars", "sum")     
).reset_index()

e_autos_per_state_2025.head()



In [ ]:
# filter the total of cars
total_cars_2025 = autos_2025 [autos_2025['fuel'] == 'Total']
total_cars_2025.drop(columns=['fuel', 'year'], inplace=True)                    
                     

Merges the total of electrical cars with the total of cars, and adds a column total of non-electric

In [ ]:
autos_state_2025 = e_autos_per_state_2025.merge(total_cars_2025, on = "state", how="left")
autos_state_2025


In [ ]:
autos_state_2025['number_non_electric_cars'] = autos_state_2025['total_cars'] - autos_state_2025['number_of_electric_cars']
autos_state_2025.rename (columns = {'total_cars':'number_cars'}, inplace = True)
autos_state_2025


In [ ]:
autos_state_2025.to_csv("./clean_data/autos_state_2025.csv", index = False)

In [ ]:
def plot_stacked_bar_for_state(df, land, colors):
    """
    Creates a stacked bar plot showing the number of electric vs. other fuel cars over different years for a given Land.

    Parameters:
        df (pd.DataFrame): The dataset containing columns 'Land', 'Fuel', 'Total cars', and 'Year'.
        land (str): The name of the Land (state) to visualize.
        german_states_colors (dict): A dictionary mapping German states to colors.
    """

    # Filter data for the selected Land
    df_land = df[df["land"] == land]

    # Pivot table to get fuel types as columns and years as index
    df_pivot = df_land.pivot_table(index="year", columns="fuel", values="total_cars", aggfunc="sum")

    # Handle missing values (if some years don't have electric cars)
    df_pivot = df_pivot.fillna(0)

    # Compute non-electric cars
    df_pivot["Non_Electric"] = df_pivot["Total"] - df_pivot["Electric"]

    # Create figure
    plt.figure(figsize=(10, 6))
    
    # Set bar width
    bar_width = 0.5  # Adjusted from default 0.8 to make bars slimmer
    
    # Plot "Non_Electric" in the state's assigned color
    plt.bar(df_pivot.index, df_pivot["Non_Electric"], color=german_states_colors.get(land, "black"), label="Other Fuels", width=bar_width)

    # Plot "Electric" on top in gray
    plt.bar(df_pivot.index, df_pivot["Electric"], bottom=df_pivot["Non_Electric"], color="gray", label="Electric", width=bar_width)


    # Add text labels for electric percentage
    for year in df_pivot.index:
        total = df_pivot.loc[year, "Total"]
        electric = df_pivot.loc[year, "Electric"]
        percent = (electric / total) * 100 if total > 0 else 0

        plt.text(year, total * 1.02, f"{percent:.1f}%", ha='center', fontsize=10, color="black")

    # Labels and title
    plt.xlabel("Year")
    plt.ylabel("Total Cars")
    plt.title(f"Electric vs Other Cars in {land} Over the Years")
    plt.xticks(df_pivot.index)  # Ensure all years are shown
    plt.legend()

    # Show the plot
    plt.show()


In [ ]:
plot_stacked_bar_for_state(df = df_states, land ='Bayern', colors= german_states_colors)

In [ ]:
plot_stacked_bar_for_state(df = df_states, land ='Nordrhein-Westfalen', colors= german_states_colors)

In [ ]:
import matplotlib.pyplot as plt

def plot_line_chart_for_state(df, land, colors):
    """
    Creates a line chart showing the number of electric vs. other fuel cars over different years for a given Land.

    Parameters:
        df (pd.DataFrame): The dataset containing columns 'Land', 'Fuel', 'Total cars', and 'Year'.
        land (str): The name of the Land (state) to visualize.
        colors (dict): A dictionary mapping fuel types to colors for the line chart.
    """

    # Filter data for the selected Land
    df_land = df[df["land"] == land]

    # Pivot table to get fuel types as columns and years as index
    df_pivot = df_land.pivot_table(index="year", columns="fuel", values="total_cars", aggfunc="sum")

    # Handle missing values (if some years don't have electric cars)
    df_pivot = df_pivot.fillna(0)

    # Compute non-electric cars
    if "Electric" in df_pivot.columns:
        df_pivot["Non_Electric"] = df_pivot["Total"] - df_pivot["Electric"]
    else:
        df_pivot["Non_Electric"] = df_pivot["Total"]  # If no electric data exists, assume all cars are non-electric

    # Create figure
    plt.figure(figsize=(10, 6))

    # Plot line chart for "Non_Electric" cars
    plt.plot(df_pivot.index, df_pivot["Non_Electric"], color=german_states_colors.get(land, "black"), label="Other Fuels", marker='o')

    # Plot line chart for "Electric" cars
    plt.plot(df_pivot.index, df_pivot["Electric"], color="gray", label="Electric", marker='o')

    # Add text labels for electric percentage on each point
    for year in df_pivot.index:
        total = df_pivot.loc[year, "Total"]
        electric = df_pivot.loc[year, "Electric"]
        percent = (electric / total) * 100 if total > 0 else 0
        plt.text(year, electric + 0.05 * total, f"{percent:.1f}%", ha='center', fontsize=10, color="black")

    # Labels and title
    plt.xlabel("Year")
    plt.ylabel("Total Cars")
    plt.title(f"Electric vs Other Cars in {land} Over the Years")
    plt.xticks(df_pivot.index)  # Ensure all years are shown
    plt.legend()

    # Show the plot
    plt.show()


In [ ]:
plot_line_chart_for_state(df = df_states, land ='Nordrhein-Westfalen', colors= german_states_colors)

In [ ]:
plot_line_chart_for_state(df = df_states, land ='Bayern', colors= german_states_colors)

In [ ]:
# Filter for electric and plug-in hybrid cars
electric_types = ["Electric", "Plug-in Hybrid"]
df_total_electric = df_states[df_states["fuel"].isin(electric_types)]

# Group by land and year to get total electric cars
electric_summary = df_total_electric.groupby(["land", "year"])["total_cars"].sum().reset_index()
electric_summary.rename(columns={"total_cars": "total_electric"}, inplace=True)



In [ ]:
# Check for rows where the fuel column is "Total"
df_total_flag = df_states[df_states["fuel"].str.lower() == "total"]

# Keep only these rows as the actual total per state/year
total_cars_summary_corrected = df_total_flag[["land", "year", "total_cars"]].copy()
total_cars_summary_corrected.rename(columns={"total_cars": "total_all"}, inplace=True)

# Reuse the already computed total electric (electric + plug-in hybrid)
# electric_summary is already grouped by land and year
# Merge with the corrected total cars
merged_df_corrected = pd.merge(total_cars_summary_corrected, electric_summary, on=["land", "year"], how="left")
merged_df_corrected["total_electric"] = merged_df_corrected["total_electric"].fillna(0).astype(int)
merged_df_corrected["total_non_electric"] = merged_df_corrected["total_all"] - merged_df_corrected["total_electric"]

# Select and reorder columns
final_df_corrected = merged_df_corrected[["land", "total_electric", "total_non_electric", "year"]]

final_df_corrected.head()


In [ ]:
final_df_corrected.shape

In [ ]:
# Set plot style
sns.set(style="whitegrid")

# Melt the DataFrame to long format for plotting
plot_df = final_df_corrected.melt(id_vars=["land", "year"],
                                  value_vars=["total_electric", "total_non_electric"],
                                  var_name="Type",
                                  value_name="Count")

# Create a color palette
palette = {"total_electric": "green", "total_non_electric": "gray"}

# Plot: Evolution per year (aggregated across all states)
plt.figure(figsize=(12, 6))
sns.barplot(data=plot_df, x="year", y="Count", hue="Type", estimator=sum, palette=palette)

plt.title("Electric vs Non-Electric Car Count Evolution in Germany")
plt.ylabel("Total Number of Cars")
plt.xlabel("Year")
plt.legend(title="Vehicle Type", labels=["Electric + Plug-in Hybrid", "Other"], bbox_to_anchor=(1.05, 1), loc="upper left")
plt.tight_layout()
plt.show()